In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
DATA_DIR = 'nyt-data'

STATE = 'California'
COUNTY = 'Orange'


country_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us.csv')),
    ],
    join='inner',
).set_index('date')
state_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us-states.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us-states.csv')),
    ],
    join='inner',
)
state_df = state_df[state_df['state'] == STATE].set_index('date')
county_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us-counties.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us-counties.csv')),
    ],
    join='inner',
)
county_df = county_df[(county_df['state'] == STATE) & (county_df['county'] == COUNTY)].set_index('date')

print('Cumulative Cases and Deaths')
print(country_df.tail())
print(state_df.tail())
print(county_df.tail())

In [ ]:
def calculate_daily_stats(cumulative_df):
    df = pd.DataFrame()
    df['cases'] = cumulative_df['cases'].diff().fillna(cumulative_df['cases'])
    df['deaths'] = cumulative_df['deaths'].diff().fillna(cumulative_df['deaths'])
    return df


country_daily_df = calculate_daily_stats(country_df)
state_daily_df = calculate_daily_stats(state_df)
county_daily_df = calculate_daily_stats(county_df)

print('Daily Cases and Deaths')
print(country_daily_df.tail())
print(state_daily_df.tail())
print(county_daily_df.tail())

In [ ]:
SIZE = (20, 12)

fig, axs = plt.subplots(3, 2, figsize=SIZE)


def draw_daily_graphs(ax_row, daily_df, region_label):
    ax_row[0].set(xticks=[], title=f'Daily Cases {region_label}')
    ax_row[0].bar(daily_df.index, daily_df['cases'])
    ax_row[0].plot(daily_df.index, daily_df['cases'].rolling(7, 1).mean(), color='red')
    
    ax_row[1].set(xticks=[], title=f'Daily Deaths {region_label}')
    ax_row[1].bar(daily_df.index, daily_df['deaths'])
    ax_row[1].plot(daily_df.index, daily_df['deaths'].rolling(7, 1).mean(), color='red')


draw_daily_graphs(axs[0], country_daily_df, 'U.S.')
draw_daily_graphs(axs[1], state_daily_df, STATE)
draw_daily_graphs(axs[2], county_daily_df, COUNTY)

plt.show()

In [ ]:
US_POPULATION = 327_167_439
STATE_POPULATION = 39_557_045
COUNTY_POPULATION = 3_185_968


def calculate_per_million_stats(cumulative_df, daily_df, population):
    df = pd.DataFrame()
    df['cases'] = 1_000_000 * cumulative_df['cases'] / population
    df['deaths'] = 1_000_000 * cumulative_df['deaths'] / population
    df['daily_cases_7_day_avg'] = 1_000_000 * daily_df['cases'].rolling(7, 1).mean() / population
    df['daily_deaths_7_day_avg'] = 1_000_000 * daily_df['deaths'].rolling(7, 1).mean() / population
    return df


def draw_per_region_graphs(ax, country_s, state_s, county_s, title):
    ax.set(xticks=[], title=title)
    ax.plot(country_s.index, country_s, color='red', label='U.S.')
    ax.plot(state_s.index, state_s, color='blue', label=STATE)
    ax.plot(county_s.index, county_s, color='green', label=COUNTY)
    ax.legend()


country_per_million_df = calculate_per_million_stats(country_df, country_daily_df, US_POPULATION)
state_per_million_df = calculate_per_million_stats(state_df, state_daily_df, STATE_POPULATION)
county_per_million_df = calculate_per_million_stats(county_df, county_daily_df, COUNTY_POPULATION)
    
SIZE = (20, 8)

fig, axs = plt.subplots(2, 2, figsize=SIZE)

draw_per_region_graphs(
    axs[0][0],
    country_per_million_df['cases'],
    state_per_million_df['cases'],
    county_per_million_df['cases'],
    'Cases per Million',
)
draw_per_region_graphs(
    axs[0][1],
    country_per_million_df['deaths'],
    state_per_million_df['deaths'],
    county_per_million_df['deaths'],
    'Deaths per Million',
)
draw_per_region_graphs(
    axs[1][0],
    country_per_million_df['daily_cases_7_day_avg'],
    state_per_million_df['daily_cases_7_day_avg'],
    county_per_million_df['daily_cases_7_day_avg'],
    'Daily Cases (7 Day Avg) per Million',
)
draw_per_region_graphs(
    axs[1][1],
    country_per_million_df['daily_deaths_7_day_avg'],
    state_per_million_df['daily_deaths_7_day_avg'],
    county_per_million_df['daily_deaths_7_day_avg'],
    'Daily Deaths (7 Day Avg) per Million',
)

plt.show()